In [1]:
%load_ext autoreload
%autoreload 2
from preprocessing import svm
from preprocessing import svm_2
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [ ]:
X_train, X_test, Y_test, Y_train = svm()
display(X_train)


Kernel polinomico

In [ ]:
metrics = []
for c in range(1, 102, 25):
    for d in range(1, 102, 25):
        for g in range(1, 102, 25):
            for r in range(1, 102, 25):
                clf = SVC(kernel='poly', C=c, degree=d, gamma=g, coef0=r, random_state=123)
                clf.fit(X_train, Y_train.values.ravel())
                acc = ((clf.predict(X_test) == Y_test.llovieron_hamburguesas_al_dia_siguiente).sum())/len(Y_test)
                metrics.append((c, d, g, r, acc))
                print(f"c {c}, d {d}, g {g}, r {r}: {acc * 100 : 3f}%")

dd = pd.DataFrame(metrics, columns=['c', 'degree', 'gamma', 'r', 'correct_in_val-dev'])

In [ ]:
df_features, df_target, pipe = svm_2()
pipe.steps.append(['svc', SVC(kernel = 'poly', random_state = 123)])

grid = GridSearchCV(pipe, param_grid={'svc__C': [1, 10, 100, 1000, 10000],
                                      'svc__degree': [1, 2],
                                      'svc__coef0': [0, 100, 1000]},
                                       cv = 3,
                                       scoring = 'accuracy',
                                       n_jobs = -1)
grid.fit(df_features, df_target.values.ravel())

In [ ]:
grid.best_score_